In [ ]:
### Tratamiento de datos
import glob
import os
import pandas as pd
import numpy as np
import calendar
from contextlib import contextmanager
from datetime import datetime
import math

###############################################################################################################################
####################################################  MAESTRAS  ###############################################################
###############################################################################################################################

os.chdir('D:/Reporting/Maestras/')
Maestro_SKU = pd.read_excel('Materiales  Abi.xlsx', sheet_name='Hoja1')#, dtype ={'Cj/Pall simil':np.float64,'HL/cj':np.float64} ,decimal='.')
Maestro_SKU.rename(columns = {'Material': 'Sku_KOA','Material ABI': 'Sku','HL/cj':'HL_cj'} ,inplace=True)
Maestro_SKU.HL_cj = np.round(Maestro_SKU.HL_cj,5)
Maestro_SKU.HL_cj =Maestro_SKU.HL_cj/10
Maestro_SKU.HL_cj = np.round(Maestro_SKU.HL_cj,6)

os.chdir('D:/Reporting/Maestras/')
Maestro_Clientes = pd.read_excel('Maestra de clientes.xlsx', sheet_name='Hoja1')
Maestro_Clientes.rename(columns = {'Cod Cliente': 'COD_ABI','Codigo Politica': 'COD_Politica'} ,inplace=True)

###############################################################################################################################
####################################################  STOCK  ##################################################################
###############################################################################################################################

############################################## STOCK KOE  #####################################################################
os.chdir('D:/Reporting/Bases Manuales/01 STOCK/Stock KOE')
Stock_KOE = []
csv_files = glob.glob('*.csv')

for filename in csv_files:
    data = pd.read_csv(filename,encoding='ANSI', sep='\t',dtype={'Stock_Cajas':np.float64, 'Sku_ABInBev': np.float64} ,decimal='.')
    Stock_KOE.append(data)
Stock_KOE = pd.concat(Stock_KOE,ignore_index=True)
Stock_KOE.Stock_Cajas = Stock_KOE.Stock_Cajas/10
Stock_KOE.Stock_Cajas = np.round(Stock_KOE.Stock_Cajas)
Stock_KOE.Sku_ABInBev = Stock_KOE.Sku_ABInBev/10
Stock_KOE.rename(columns = {'Nom_cd': 'CD','Sku_ABInBev': 'Sku', 'A¤o_Mes_Dia': 'Fecha_t','Stock_Cajas': 'Stock_Cj'} ,inplace=True)
Stock_KOE=Stock_KOE[["CD","Sku","Stock_Cj","Fecha_t"]]
Stock_KOE = Stock_KOE[Stock_KOE["Stock_Cj"]>0]
Stock_KOE["PRI"]=0

################################################ STOCK KOA  ####################################################################
os.chdir('D:/Reporting/Bases Manuales/01 STOCK/Stock KOA')
Stock_KOA = []
csv_files = glob.glob('*.csv')

for filename in csv_files:
    data = pd.read_csv(filename,encoding='ANSI', sep=';',dtype={'Stock':np.float64,'Codigo CD': np.float64} ,decimal='.')
    data['filename']=filename
    Stock_KOA.append(data)
Stock_KOA = pd.concat(Stock_KOA,ignore_index=True)

Titulo =Stock_KOA[["filename"]]
Titulo = Titulo["filename"].str.split(" ", n=1, expand=True)
Stock_KOA["Fecha_t"]=Titulo[0].str.replace(r"-","")
Stock_KOA.rename(columns = {'Codigo CD': 'CD','SKU': 'Sku_KOA', 'Stock': 'Stock_Cj'} ,inplace=True)
Stock_KOA.Sku_KOA = Stock_KOA.Sku_KOA.astype(int)

Maestra_SKU_KOA = Maestro_SKU
Maestra_SKU_KOA.Sku_KOA = Maestra_SKU_KOA.Sku_KOA.astype(int)

Maestra_SKU_KOA = Maestro_SKU[["Sku_KOA","Sku"]]
Maestra_SKU_KOA = Maestra_SKU_KOA.drop_duplicates(subset=["Sku_KOA"])
Stock_KOA = pd.merge(Stock_KOA,Maestra_SKU_KOA,on=["Sku_KOA"],how="outer")
#Stock_KOA.CD = Stock_KOA.CD/10
Stock_KOA.Sku = Stock_KOA.Sku.astype(np.float64)
Stock_KOA.Sku = Stock_KOA.Sku/10
Stock_KOA.Stock_Cj = Stock_KOA.Stock_Cj/10
Stock_KOA["PRI"] =0
Stock_KOA=Stock_KOA[["CD","Sku","Stock_Cj","Fecha_t","PRI"]]
Stock_KOA = Stock_KOA.dropna(subset=['Fecha_t'])


################################################# STOCK MAQUILA  ###############################################################
os.chdir('D:/Reporting/Bases Manuales/01 STOCK/Stock ABI/Stock Maquila')
Stock_ABI_Maq = []
csv_files = glob.glob('*.txt')

for filename in csv_files:
    data = pd.read_csv(filename,encoding='ANSI', sep=',',dtype={' Stock     ':np.float64}, decimal='.')
    data['filename']=filename
    Stock_ABI_Maq.append(data)
Stock_ABI_Maq = pd.concat(Stock_ABI_Maq,ignore_index=True)

Titulo =Stock_ABI_Maq[["filename"]]
Titulo = Titulo["filename"].str.split(" ", n=1, expand=True)
Stock_ABI_Maq["Fecha_t"]=Titulo[0].str.replace(r"-","")
Stock_ABI_Maq['Base']="Maq"
Stock_ABI_Maq.rename(columns = {'Deposito ': 'CD', ' Stock     ': 'Stock_Cj', '   Cod.Pro': 'Sku'} ,inplace=True)
Stock_ABI_Maq = Stock_ABI_Maq[Stock_ABI_Maq.CD.isin([65, 70])]
Stock_ABI_Maq["PRI"] = 0

Stock_ABI_Maq =Stock_ABI_Maq[["CD","Sku","Stock_Cj","Fecha_t","PRI"]]

Stock_ABI_Maq.Stock_Cj = Stock_ABI_Maq.Stock_Cj/10
Stock_ABI_Maq.Stock_Cj = np.round(Stock_ABI_Maq.Stock_Cj)
Stock_ABI_Maq = Stock_ABI_Maq[Stock_ABI_Maq["Stock_Cj"]>0]

################################################### STOCK VENTA  #############################################################
os.chdir('D:/Reporting/Bases Manuales/01 STOCK/Stock ABI/Stock Venta')
Stock_ABI_Ven = []
csv_files = glob.glob('*.txt')

for filename in csv_files:
    data = pd.read_csv(filename,encoding='ANSI', sep=',',dtype={' Stock     ':np.float64} ,decimal='.')
    data['filename']=filename
    Stock_ABI_Ven.append(data)
Stock_ABI_Ven = pd.concat(Stock_ABI_Ven,ignore_index=True)

Titulo =Stock_ABI_Ven[["filename"]]
Titulo = Titulo["filename"].str.split(" ", n=1, expand=True)
Stock_ABI_Ven["Fecha_t"]=Titulo[0].str.replace(r"-","")
Stock_ABI_Ven['Base']="Ven"
Stock_ABI_Ven.rename(columns = {'Deposito ': 'CD', ' Stock     ': 'Stock', '   Cod.Pro': 'Sku', ' Transito ':'Transito',' PRI - 100 ':'PRI'} ,inplace=True)
Stock_ABI_Ven['Stock_Cj']= Stock_ABI_Ven.Stock+Stock_ABI_Ven.Transito
Stock_ABI_Ven.PRI = Stock_ABI_Ven.PRI/10
Stock_ABI_Ven.Stock_Cj = np.round(Stock_ABI_Ven.Stock_Cj)
Stock_ABI_Ven.PRI = np.round(Stock_ABI_Ven.PRI)
Stock_ABI_Ven = Stock_ABI_Ven[Stock_ABI_Ven["Stock_Cj"]>0]
Stock_ABI_Ven.Stock_Cj = Stock_ABI_Ven.Stock_Cj/10
Stock_ABI_Ven = Stock_ABI_Ven[["CD","Sku","Stock_Cj","Fecha_t","PRI"]]

###### MAESTRAS ####
os.chdir('D:/Reporting/Maestras/')
Maestro_SKU = pd.read_excel('Materiales  Abi.xlsx', sheet_name='Hoja1')#, dtype ={'Cj/Pall simil':np.float64,'HL/cj':np.float64} ,decimal='.')
Maestro_SKU.rename(columns = {'Material': 'Sku_KOA','Material ABI': 'SKU','HL/cj':'HL_cj'} ,inplace=True)
Maestro_SKU.HL_cj = np.round(Maestro_SKU.HL_cj,5)
Maestro_SKU.HL_cj =Maestro_SKU.HL_cj/10
Maestro_SKU.HL_cj = np.round(Maestro_SKU.HL_cj,6)
Maestro_SKU = Maestro_SKU[["SKU","SKU KOE"]]
Maestro_SKU = Maestro_SKU.drop_duplicates(['SKU KOE'])

os.chdir('D:/Reporting/Maestras/')
Maestro_Clientes = pd.read_excel('Maestra de clientes.xlsx', sheet_name='Hoja1')
Maestro_Clientes.rename(columns = {'CP STOCK': 'CD','Codigo Stock Manual':'CD_KOE'} ,inplace=True)
Mt_Cli2 = Maestro_Clientes[["CD","CD_KOE"]]

##################################   Stock KOE 2   ######################################

os.chdir('D:/Reporting/Bases Manuales/01 STOCK/Stock KOE real')
Stock_KOE_2 = []
csv_files = glob.glob('*.xlsx')

for filename in csv_files:
    data = pd.read_excel(filename,sheet_name='Base SAP')#, dtype={'Volumen en HL':np.float64})
    data['filename']=filename
    Stock_KOE_2.append(data)
Stock_KOE_2 = pd.concat(Stock_KOE_2,ignore_index=True)

T1 = Stock_KOE_2[["filename"]]
T1 = T1["filename"].str.split(".", n=1, expand=True)
Stock_KOE_2['Fecha_t'] = T1[0]
Stock_KOE_2 = Stock_KOE_2[["Fecha_t","Nombre 1","Material","Cajas Stock"]]
Stock_KOE_2.rename(columns = {'Nombre 1': 'CD_KOE', 'Material': 'SKU KOE','Cajas Stock':'Stock_Cj'} ,inplace=True)

Stock_KOE_2 = Stock_KOE_2[Stock_KOE_2.Stock_Cj!=0]
Stock_KOE_2.Stock_Cj = Stock_KOE_2.Stock_Cj.astype(str)

Stock_KOE_2.Stock_Cj = Stock_KOE_2.Stock_Cj.str.replace(r"''","0")
Stock_KOE_2.Stock_Cj = Stock_KOE_2.Stock_Cj.str.replace(r" ","0")
Stock_KOE_2.Stock_Cj = Stock_KOE_2.Stock_Cj.str.replace(r",","0")
Stock_KOE_2.Stock_Cj = Stock_KOE_2.Stock_Cj.astype(float)
#Stock_KOE_2 = Stock_KOE_2[Stock_KOE_2.Cj>0]
Stock_KOE_2.Stock_Cj = np.round(Stock_KOE_2.Stock_Cj,0)

Maestro_SKU = Maestro_SKU.drop_duplicates(subset=["SKU KOE"])
Stock_KOE_2 = pd.merge(Stock_KOE_2,Maestro_SKU,on=["SKU KOE"],how="outer")
Stock_KOE_2 = pd.merge(Stock_KOE_2,Mt_Cli2,on=["CD_KOE"],how="outer")

Stock_KOE_2.SKU = Stock_KOE_2.SKU.astype(str)
T1 = Stock_KOE_2[["SKU"]]
T1 = T1["SKU"].str.split(".", n=1, expand=True)
Stock_KOE_2['SKU'] = T1[0]
Stock_KOE_2 = Stock_KOE_2.dropna(subset=['Fecha_t'])
Stock_KOE_2.rename(columns = {'SKU': 'Sku'} ,inplace=True)
Stock_KOE_2 = Stock_KOE_2[["Fecha_t","CD","Sku","Stock_Cj"]]

Stock_KOE_2.Stock_Cj = Stock_KOE_2.Stock_Cj/10
#Stock_KOE_2.Stock_Cj = np.round(Stock_KOE_2.Stock_Cj,0)
Stock_KOE_2.PRI=0

############################################# STOCK SAP ####################################################
os.chdir('D:/Reporting/Bases Manuales/01 STOCK/Stock ABI/Stock SAP')
DF = []
csv_files = glob.glob('*.txt')

for filename in csv_files:
    data = pd.read_csv(filename,encoding='ANSI', sep='\t')#,dtype={' Libre utilización':np.float64} ,decimal='.')
    data['filename']=filename
    xx = data.filename.str.split(".", n=1, expand=True)
    data['Fecha_t'] = xx[0]
    DF.append(data)
DF = pd.concat(DF,ignore_index=True)
DF = DF.dropna(subset=['Material'])
DF.Material = DF.Material.astype(str)
DF.Material = DF.Material.str.split(".", n=1, expand=True)
DF.rename(columns = {'Material':'SKU_SAP','Ce.': 'Cent','Alm.':'CD_SAP',' Libre utilización':'Cj',' Valor stock bloq.':'PRI'} ,inplace=True)
DF = DF[["SKU_SAP","CD_SAP","Cj","PRI","Fecha_t"]]

os.chdir('D:/Reporting/Maestras/')
Maestro_CD = pd.read_excel('Maestras SAP.xlsx', sheet_name='CD')
Maestro_CD.CD_SAP = Maestro_CD.CD_SAP.astype(str)
Maestro_CD.CD = Maestro_CD.CD.astype(str)

os.chdir('D:/Reporting/Maestras/')
Maestro_SKU = pd.read_excel('Maestras SAP.xlsx', sheet_name='SKU_SAP')
Maestro_SKU
Maestro_SKU.Cod_truck = Maestro_SKU.Cod_truck.astype(str)
Maestro_SKU.SKU_SAP = Maestro_SKU.SKU_SAP.astype(str)
Maestro_SKU.SKU_SAP = Maestro_SKU.SKU_SAP.str.split(".", n=1, expand=True)
Maestro_SKU = Maestro_SKU[["SKU_SAP","Cod_truck"]]

DF = pd.merge(DF,Maestro_CD,on=["CD_SAP"],how="outer")
DF = pd.merge(DF,Maestro_SKU,on=["SKU_SAP"],how="outer")
DF = DF.dropna(subset=['Fecha_t'])
DF = DF[["CD","Cod_truck" ,"Cj","Fecha_t","PRI"]]
DF.rename(columns = {'Cod_truck':'Sku','Cj': 'Stock_Cj'} ,inplace=True)


############################   EXPORTABLES  ############################################
Stock_KO = Stock_KOE.append(Stock_KOA)
Stock_KO = Stock_KO.append(Stock_KOE_2)
Stock_ABI = Stock_ABI_Ven.append(Stock_ABI_Maq)
Stock_ABI = Stock_ABI.append(DF)


Stock_KO = Stock_KO.dropna(subset=['Fecha_t','CD','Sku'])
Stock_KO.to_csv("D:/Reporting/ETL/02 STOCK/Stock KO/Stock_KO.txt")
Stock_ABI.to_csv("D:/Reporting/ETL/02 STOCK/Stock ABI/Stock_ABI.txt")

###############################################################################################################################
#####################################################   SELL   ################################################################
###############################################################################################################################

#### SELL IN
os.chdir('D:/Reporting/Bases Manuales/02 SELL/Sell IN')
Sell_in = []
csv_files = glob.glob('*.txt')

for filename in csv_files:
    data = pd.read_csv(filename,encoding='UTF-8', sep='\t')
    data['filename']=filename
    Sell_in.append(data)
Sell_in = pd.concat(Sell_in,ignore_index=True)
Sell_in.rename(columns = {'Año Mes Dia Emision':'Fecha_SI','C.Dep.Ori': 'DPS ORI','C.Prd':'Sku', 'N.Planilla': 'PLANILLA', 'Camion Patente':'COD FLETERO'} ,inplace=True)
Sell_in= Sell_in[["Fecha_SI","Año Mes Dia Cierre","C.Cli","PLANILLA","COD FLETERO",'Sku','#Cj Vta Bruta SUM','#HL Vta Bruta SUM','D.Est.Pla']]

Sell_in_WPS = Sell_in
Sell_in_WPS.rename(columns ={'#Cj Vta Bruta SUM':'Cj', 'C.Cli': 'DPS DES'},inplace=True)


###### EXPORTABLE
Sell_in
Sell_in.to_csv("D:/Reporting/ETL/01 SELL/Sell In/Sell_In.txt")

#############################################################################################
########################################## Politica #########################################
#############################################################################################

os.chdir('D:/Reporting/Bases Manuales/03 POLITICA/Diaria')
Politica = []
csv_files = glob.glob('*.xlsx')

for filename in csv_files:
    data = pd.read_excel(filename,sheet_name='Hoja1')#, dtype={'FCST HL ajustado':np.float64,'Seg HL':np.float64, 'Repo HL':np.float64, 'Max HL':np.float64, 'OOR Out HL':np.float64, 'OOR Over HL':np.float64} ,decimal=',')
    data['filename']=filename
    Politica.append(data)
Politica = pd.concat(Politica,ignore_index=True)

Titulo = Politica[["filename"]]
Titulo = Titulo["filename"].str.split(".", n=1, expand=True)
Politica["Fecha_t"]=Titulo[0].str.replace(r".","")

Politica.rename(columns ={'FCST HL ajustado':'FCST_HL', 'Seg HL': 'Seg_HL', 'Repo HL': 'Repo_HL', 'Max HL': 'Max_HL', 'OOR Out HL': 'OOR_Out_HL', 'OOR Over HL': 'OOR_Over_HL'},inplace=True)
# Politica.FCST_HL = np.round(Politica.FCST_HL,1)
Politica.Seg_HL = np.round(Politica.Seg_HL,1)
Politica.Repo_HL = np.round(Politica.Repo_HL,1)
Politica.Max_HL = np.round(Politica.Max_HL,1)
Politica.OOR_Out_HL = np.round(Politica.OOR_Out_HL,1)
Politica.OOR_Over_HL = np.round(Politica.OOR_Over_HL,1)

Politica = Politica[['Fecha_t', 'Region','Cod. Producto','FCST_HL','Seg_HL', 'Repo_HL', 'Max_HL', 'OOR_Out_HL', 'OOR_Over_HL','Transito Hl','Cargan hoy Hl','D+1  Hl','DV centro','Stock centro hoy HL']]
Politica.rename(columns ={'Transito Hl':'Transito','Cargan hoy Hl':'Cargan_hoy','D+1  Hl':'D1','Stock centro hoy HL':'Stock_centro_hoy_HL','DV centro':'DV_centro'},inplace=True)

Politica.Cargan_hoy = np.round(Politica.Cargan_hoy,1)
Politica.Transito = np.round(Politica.Transito,1)
Politica.Stock_centro_hoy_HL = np.round(Politica.Stock_centro_hoy_HL,1)
#Politica.DV_centro = np.round(Politica.DV_centro,1)
Politica.D1 = np.round(Politica.D1,1)

os.chdir('D:/Reporting/Bases Manuales/03 POLITICA/Nuevo formato')
Politica2 = []
csv_files = glob.glob('*.xlsx')

for filename in csv_files:
    data = pd.read_excel(filename,sheet_name='Hoja1')#, dtype={'FCST_HL':np.float64,'Seg_HL':np.float64, 'Repo_HL':np.float64, 'Max_HL':np.float64, 'OOR_OUT_HL':np.float64, 'OOR_OVER_HL':np.float64} ,decimal=',')
    data['filename']=filename
    Politica2.append(data)
Politica2 = pd.concat(Politica2,ignore_index=True)

Titulo = Politica2[["filename"]]
Titulo = Titulo["filename"].str.split(".", n=1, expand=True)
Politica2["Fecha_t"]=Titulo[0].str.replace(r".","")

Politica2.Seg_HL = np.round(Politica2.Seg_HL,1)#Seg_HL
Politica2.Repo_HL = np.round(Politica2.Repo_HL,1)#Repo_HL
Politica2.Max_HL = np.round(Politica2.Max_HL,1)#Max_HL
Politica2.OOR_OUT_HL = np.round(Politica2.OOR_OUT_HL,1)#OOR_OUT_HL
Politica2.OOR_OVER_HL = np.round(Politica2.OOR_OVER_HL,1)#OOR_OVER_HL

Politica2.D0_PLL = np.round(Politica2.D0_PLL,1)#D0_PLL
Politica2.Transito_PLL = np.round(Politica2.Transito_PLL,1)#Transito_PLL
Politica2.D1_PLL = np.round(Politica2.D1_PLL,1)#D1_PLL
Politica2.SugT_HL = np.round(Politica2.SugT_HL,1)#SugT_HL
Politica2.SugR_HL = np.round(Politica2.SugR_HL,1)#SugR_HL
Politica2.Dispo_HL = np.round(Politica2.Dispo_HL,1)#Dispo_HL
Politica2.Stock_Py_HL = np.round(Politica2.Stock_Py_HL,1)#Stock_Py_HL
Politica2.Stock_Tron_HL = np.round(Politica2.Stock_Tron_HL,1)#Stock_Tron_HL
Politica2.FCST_HL = np.round(Politica2.FCST_HL,1)#FCST_HL
Politica2.Dispo_ABI_HL = np.round(Politica2.Dispo_ABI_HL,1)#Dispo_ABI_HL
Politica2.Dispo_Centro_HL = np.round(Politica2.Dispo_Centro_HL,1)# Dispo_Centro_HL 

Politica2 = Politica2[['Fecha_t', 'Region','Cod. Producto','FCST_HL','Seg_HL', 'Repo_HL', 'Max_HL', 'OOR_OUT_HL', 'OOR_OVER_HL','Transito_PLL','D0_PLL','D1_PLL','Dispo_Centro_HL']]
Politica2.rename(columns ={'OOR_OUT_HL':'OOR_Out_HL','OOR_OVER_HL':'OOR_Over_HL', 'Transito_PLL': 'Transito', 'D0_PLL':'Cargan_hoy', 'D1_PLL': 'D1', 'Dispo_ABI_HL':'Stock_centro_hoy_HL'},inplace=True)
Politica2 = Politica2.fillna(0)

Politica = Politica.append(Politica2)

#### EXPORTABLE
Politica.to_csv("D:/Reporting/ETL/03 Politica/Politica.txt")
################## STOCK SEMANAL #####################


In [ ]:
Politica

In [ ]:
Politica2

In [ ]:
################### FIN #########################

In [ ]:
Stock_KOA.Sku = Stock_KOA.Sku*10
Stock_KOA.Sku = Stock_KOA.Sku.astype(str)
t = Stock_KOA.Sku.str.split(".", n=1, expand=True)
Stock_KOA.Sku = t[0]
Stock_KOA = Stock_KOA.drop(Stock_KOA[Stock_KOA['Sku']=='nan'].index)
Stock_KOA['Id'] = "Andina" +"-"+ Stock_KOA.Fecha_t.astype(str) +"-"+Stock_KOA.Sku.astype(str)
Stock_KOA_gb = Stock_KOA.groupby(by = ["Id"])["Stock_Cj"].sum().reset_index()
Stock_KOA_gb = pd.DataFrame(Stock_KOA_gb)
t = Stock_KOA_gb.Id.str.split("-", n=2, expand=True)
Stock_KOA_gb['Bottler'] = t[0]
Stock_KOA_gb['Fecha'] = t[1]
Stock_KOA_gb['Sku'] = t[2]
Stock_KOA_gb = Stock_KOA_gb[Stock_KOA_gb["Stock_Cj"]>0]

Stock_KOA_gb = Stock_KOA_gb.drop(Stock_KOA_gb[Stock_KOA_gb['Fecha']==' 092905'].index)
Stock_KOA_gb.Fecha = Stock_KOA_gb.Fecha.astype(float)
Stock_KOA_gb.Fecha = Stock_KOA_gb.Fecha.round(0)
Stock_KOA_gb = Stock_KOA_gb[Stock_KOA_gb["Fecha"]<20239999]
Stock_KOA_gb['Fecha_f'] = pd.to_datetime(Stock_KOA_gb['Fecha'], format="%Y%m%d")
Stock_KOA_gb['weekday'] = Stock_KOA_gb['Fecha_f'].dt.dayofweek
Stock_KOA_gb_L = Stock_KOA_gb[Stock_KOA_gb["weekday"]==0]
Stock_KOA_gb_L = Stock_KOA_gb_L[["Id", "Stock_Cj"]]
Stock_KOA_gb_L

Stock_KOA_gb_L.Stock_Cj = Stock_KOA_gb_L.Stock_Cj.astype(float)
Stock_KOA_gb_L.Stock_Cj = Stock_KOA_gb_L.Stock_Cj.round(3)
Stock_KOA_gb_L.Stock_Cj = Stock_KOA_gb_L.Stock_Cj*1000
Stock_KOA_gb_L.Stock_Cj = Stock_KOA_gb_L.Stock_Cj.round(0)

In [ ]:
######## STOCK DECK #######
################ KOA ############
os.chdir('D:/Reporting/Maestras/')
Maestro_SKU = pd.read_excel('Materiales  Abi.xlsx', sheet_name='Hoja1')#, dtype ={'Cj/Pall simil':np.float64,'HL/cj':np.float64} ,decimal='.')
Maestro_SKU.rename(columns = {'Material': 'Sku_KOA','Material ABI': 'Sku','HL/cj':'HL_cj'} ,inplace=True)
Maestro_SKU.HL_cj = np.round(Maestro_SKU.HL_cj,5)
Maestro_SKU.HL_cj =Maestro_SKU.HL_cj/10
Maestro_SKU.HL_cj = np.round(Maestro_SKU.HL_cj,6)

################################################ STOCK KOA  ####################################################################
os.chdir('D:/Reporting/Bases Manuales/01 STOCK/Stock KOA')
Stock_KOA = []
csv_files = glob.glob('*.csv')

for filename in csv_files:
    data = pd.read_csv(filename,encoding='ANSI', sep=';',dtype={'Stock':np.float64,'Codigo CD': np.float64} ,decimal='.')
    data['filename']=filename
    Stock_KOA.append(data)
Stock_KOA = pd.concat(Stock_KOA,ignore_index=True)

Titulo =Stock_KOA[["filename"]]
Titulo = Titulo["filename"].str.split(" ", n=1, expand=True)
Stock_KOA["Fecha_t"]=Titulo[0].str.replace(r"-","")
Stock_KOA.rename(columns = {'Codigo CD': 'CD','SKU': 'Sku_KOA', 'Stock': 'Stock_Cj'} ,inplace=True)
Stock_KOA.Sku_KOA = Stock_KOA.Sku_KOA.astype(int)

Maestra_SKU_KOA = Maestro_SKU
Maestra_SKU_KOA.Sku_KOA = Maestra_SKU_KOA.Sku_KOA.astype(int)

Maestra_SKU_KOA = Maestro_SKU[["Sku_KOA","Sku"]]
Maestra_SKU_KOA = Maestra_SKU_KOA.drop_duplicates(subset=["Sku_KOA"])
Stock_KOA = pd.merge(Stock_KOA,Maestra_SKU_KOA,on=["Sku_KOA"],how="outer")
#Stock_KOA.CD = Stock_KOA.CD/10
Stock_KOA.Sku = Stock_KOA.Sku.astype(np.float64)

Stock_KOA["PRI"] =0
Stock_KOA=Stock_KOA[["CD","Sku","Stock_Cj","Fecha_t","PRI"]]
Stock_KOA = Stock_KOA.dropna(subset=['Fecha_t'])

Stock_KOA.Sku = Stock_KOA.Sku.astype(str)
t = Stock_KOA.Sku.str.split(".", n=1, expand=True)
Stock_KOA.Sku = t[0]
Stock_KOA = Stock_KOA.drop(Stock_KOA[Stock_KOA['Sku']=='nan'].index)
###### Resumen DECK
Stock_KOA['Id'] = "Andina" +"-"+ Stock_KOA.Fecha_t.astype(str) +"-"+Stock_KOA.Sku.astype(str)
Stock_KOA_gb = Stock_KOA.groupby(by = ["Id"])["Stock_Cj"].sum().reset_index()
Stock_KOA_gb = pd.DataFrame(Stock_KOA_gb)
t = Stock_KOA_gb.Id.str.split("-", n=2, expand=True)
Stock_KOA_gb['Bottler'] = t[0]
Stock_KOA_gb['Fecha'] = t[1]
Stock_KOA_gb['Sku'] = t[2]
Stock_KOA_gb = Stock_KOA_gb[Stock_KOA_gb["Stock_Cj"]>0]

Stock_KOA_gb = Stock_KOA_gb.drop(Stock_KOA_gb[Stock_KOA_gb['Fecha']==' 092905'].index)
Stock_KOA_gb.Fecha = Stock_KOA_gb.Fecha.astype(float)
Stock_KOA_gb.Fecha = Stock_KOA_gb.Fecha.round(0)
Stock_KOA_gb = Stock_KOA_gb[Stock_KOA_gb["Fecha"]<20239999]
Stock_KOA_gb['Fecha_f'] = pd.to_datetime(Stock_KOA_gb['Fecha'], format="%Y%m%d")
Stock_KOA_gb['weekday'] = Stock_KOA_gb['Fecha_f'].dt.dayofweek
###### FILTRO DIA LUNES
Stock_KOA_gb_L = Stock_KOA_gb[Stock_KOA_gb["weekday"]==0]
Stock_KOA_gb_L = Stock_KOA_gb_L[["Id", "Stock_Cj"]]

Stock_KOA_gb_L.Stock_Cj = Stock_KOA_gb_L.Stock_Cj.astype(float)
Stock_KOA_gb_L.Stock_Cj = Stock_KOA_gb_L.Stock_Cj.round(3)
Stock_KOA_gb_L.Stock_Cj = Stock_KOA_gb_L.Stock_Cj*1000
Stock_KOA_gb_L.Stock_Cj = Stock_KOA_gb_L.Stock_Cj.round(0)
Stock_KOA_gb_L.to_csv("D:/BI Plannig/02 ETL/02 DATA/07 Stock/Stock_KOA.txt")

################### KOE #########################################################
os.chdir('D:/Reporting/Bases Manuales/01 STOCK/Stock KOE real')
Stock_KOE_2 = []
csv_files = glob.glob('*.xlsx')

for filename in csv_files:
    data = pd.read_excel(filename,sheet_name='Base SAP')#, dtype={'Volumen en HL':np.float64})
    data['filename']=filename
    Stock_KOE_2.append(data)
Stock_KOE_2 = pd.concat(Stock_KOE_2,ignore_index=True)

T1 = Stock_KOE_2[["filename"]]
T1 = T1["filename"].str.split(".", n=1, expand=True)
Stock_KOE_2['Fecha_t'] = T1[0]
Stock_KOE_2 = Stock_KOE_2[["Fecha_t","Nombre 1","Material","Cajas Stock"]]
Stock_KOE_2.rename(columns = {'Nombre 1': 'CD_KOE', 'Material': 'SKU KOE','Cajas Stock':'Stock_Cj'} ,inplace=True)

### STOCK
Stock_KOE_2 = Stock_KOE_2[Stock_KOE_2.Stock_Cj!=0]
Stock_KOE_2.Stock_Cj = Stock_KOE_2.Stock_Cj.astype(str)
Stock_KOE_2.Stock_Cj = Stock_KOE_2.Stock_Cj.str.replace(r"''","0")
Stock_KOE_2.Stock_Cj = Stock_KOE_2.Stock_Cj.str.replace(r" ","0")
Stock_KOE_2.Stock_Cj = Stock_KOE_2.Stock_Cj.str.replace(r",","0")
Stock_KOE_2.Stock_Cj = Stock_KOE_2.Stock_Cj.astype(float)
#Stock_KOE_2 = Stock_KOE_2[Stock_KOE_2.Cj>0]
Stock_KOE_2.Stock_Cj = np.round(Stock_KOE_2.Stock_Cj,0)

### SKU
Maestro_SKU = Maestro_SKU.drop_duplicates(subset=["SKU KOE"])
Stock_KOE_2 = pd.merge(Stock_KOE_2,Maestro_SKU,on=["SKU KOE"],how="outer")
Stock_KOE_2 = pd.merge(Stock_KOE_2,Mt_Cli2,on=["CD_KOE"],how="outer")
Stock_KOE_2.SKU = Stock_KOE_2.SKU.astype(str)
T1 = Stock_KOE_2[["SKU"]]
T1 = T1["SKU"].str.split(".", n=1, expand=True)
Stock_KOE_2['SKU'] = T1[0]
### fECHA
Stock_KOE_2 = Stock_KOE_2.dropna(subset=['Fecha_t'])
Stock_KOE_2.rename(columns = {'SKU': 'Sku'} ,inplace=True)
Stock_KOE_2 = Stock_KOE_2[["Fecha_t","CD","Sku","Stock_Cj"]]

#########################################################################################################################
####### Agrupar sin CD
Stock_KOE_2['Id'] = "Embonor" +"-"+ Stock_KOE_2.Fecha_t.astype(str) +"-"+Stock_KOE_2.Sku.astype(str)
Stock_KOE_gb = Stock_KOE_2.groupby(by = ["Id"])["Stock_Cj"].sum().reset_index()
Stock_KOE_gb = pd.DataFrame(Stock_KOE_gb)
t = Stock_KOE_gb.Id.str.split("-", n=2, expand=True)
#######
Stock_KOE_gb['Bottler'] = t[0]
Stock_KOE_gb['Fecha'] = t[1]
Stock_KOE_gb['Sku'] = t[2]

Stock_KOE_gb['Fecha_f'] = pd.to_datetime(Stock_KOE_gb['Fecha'], format="%Y%m%d")
Stock_KOE_gb['weekday'] = Stock_KOE_gb['Fecha_f'].dt.dayofweek
##### FILTRO DEL DIA LUNES
Stock_KOE_gb_L = Stock_KOE_gb[Stock_KOE_gb["weekday"]==0]
Stock_KOE_gb_L = Stock_KOE_gb_L[Stock_KOE_gb_L["Sku"]!="nan"]
Stock_KOE_gb_L = Stock_KOE_gb_L[["Id", "Stock_Cj"]]

Stock_KOE_gb_L.Stock_Cj = Stock_KOE_gb_L.Stock_Cj.astype(float)
Stock_KOE_gb_L.Stock_Cj = Stock_KOE_gb_L.Stock_Cj.round(3)
Stock_KOE_gb_L.Stock_Cj = Stock_KOE_gb_L.Stock_Cj*1000
Stock_KOE_gb_L.Stock_Cj = Stock_KOE_gb_L.Stock_Cj.round(0)

#########################################################################################################################
############ ABI #############
os.chdir('D:/Reporting/Bases Manuales/01 STOCK\Stock ABI/Control de Stock')
Stock_CS = []
csv_files = glob.glob('*.xlsx')

for filename in csv_files:
    data = pd.read_excel(filename,sheet_name='Base')#, dtype={'FCST HL ajustado':np.float64,'Seg HL':np.float64, 'Repo HL':np.float64, 'Max HL':np.float64, 'OOR Out HL':np.float64, 'OOR Over HL':np.float64} ,decimal=',')
    data['filename']=filename
    Stock_CS.append(data)
Stock_CS = pd.concat(Stock_CS,ignore_index=True)
Stock_CS.rename(columns = {'Ce.': 'CD','Codigo SAP': 'SKU_SAP','Stock final':'Stock_Cj'} ,inplace=True)

###E2 Atributos
# SKU
os.chdir('D:/Reporting/Maestras/SKU Oficial')
Maestro_SKU = pd.read_excel('Maestra ABI Oficial.xlsx', sheet_name='Hoja1')#, dtype ={'Cj/Pall simil':np.float64,'HL/cj':np.float64} ,decimal='.')
Maestro_SKU = Maestro_SKU.drop(['Unnamed: 0'], axis=1)
Maestro_SKU.columns = Maestro_SKU.iloc[0]
Maestro_SKU  = Maestro_SKU.drop([0],axis=0)
Maestro_SKU.rename(columns = {'Código truck': 'SKU','Codigo SAP': 'SKU_SAP'} ,inplace=True)
Maestro_SKU_KO = Maestro_SKU[["SKU", "SKU_SAP"]]
Maestro_SKU_KO = Maestro_SKU_KO.dropna(subset=['SKU_SAP'])
Maestro_SKU_KO = Maestro_SKU_KO[Maestro_SKU_KO["SKU_SAP"]!="nan"]
Maestro_SKU_KO = Maestro_SKU_KO[Maestro_SKU_KO["SKU_SAP"]!="NaN"]
Maestro_SKU_KO.SKU_SAP = Maestro_SKU_KO.SKU_SAP.astype(int)

Stock_CS.SKU_SAP = Stock_CS.SKU_SAP.astype(str)
Titulo = Stock_CS["SKU_SAP"].str.split(".", n=1, expand=True)
Stock_CS['SKU_SAP'] =  Titulo[0].astype(str)# + Titulo[1].astype(str) + Titulo[2].astype(str)
Stock_CS
#Stock_CS = Stock_CS[["filename", "SKU_SAP", "Stock cajas", "HLs"]]
Stock_CS = Stock_CS.dropna(subset=['SKU_SAP'])
Stock_CS = Stock_CS[Stock_CS["SKU_SAP"]!="nan"]
Stock_CS.SKU_SAP = Stock_CS.SKU_SAP.astype(int)

Stock_CS = pd.merge(Stock_CS, Maestro_SKU_KO, on=["SKU_SAP"],how="outer")
#Stock_CS.rename(columns = {'Stock cajas': 'SKU','Codigo SAP': 'SKU_SAP'} ,inplace=True)
# HL
Stock_CS.HLs = Stock_CS.HLs.astype(float)
Stock_CS.HLs = Stock_CS.HLs.round(3)
Stock_CS.HLs = Stock_CS.HLs*1000
Stock_CS.HLs = Stock_CS.HLs.round(0)
#CJ
Stock_CS.Stock_Cj = Stock_CS.Stock_Cj.astype(float)
Stock_CS.Stock_Cj = Stock_CS.Stock_Cj.round(3)
Stock_CS.Stock_Cj = Stock_CS.Stock_Cj*1000
Stock_CS.Stock_Cj = Stock_CS.Stock_Cj.round(0)
#Fecha
Stock_CS.filename = Stock_CS.filename.astype(str)
Titulo = Stock_CS["filename"].str.split(".", n=1, expand=True)
Stock_CS['Fecha'] =  Titulo[0].astype(str)# + Titulo[1].astype(str) + Titulo[2].astype(str)
Stock_CS = Stock_CS.dropna(subset=['SKU_SAP','Fecha'])
Stock_CS.Fecha = Stock_CS.Fecha.astype(str)

Stock_CS_gb = Stock_CS.groupby(by = ["Fecha", "SKU_SAP", "SKU"])["Stock_Cj", "HLs"].sum().reset_index()
Stock_CS_gb = pd.DataFrame(Stock_CS_gb)
Stock_CS_gb = Stock_CS.dropna(subset=['SKU_SAP','Fecha'])

Stock_CS_gb['Fecha_d'] = Stock_CS_gb.Fecha.str.extract(r"(\d{2})") 
Stock_CS_gb['Fecha_dm'] = Stock_CS_gb.Fecha.str.extract(r"(\d{4})").astype(str)
Stock_CS_gb['Fecha_m'] = Stock_CS_gb.Fecha_dm.apply(lambda x: x[2:])
Stock_CS_gb['Fecha_A'] = Stock_CS_gb.Fecha.apply(lambda x: x[4:]) 
Stock_CS_gb['Fecha_t'] = Stock_CS_gb.Fecha_A + Stock_CS_gb.Fecha_m + Stock_CS_gb.Fecha_d
Stock_CS_gb['Fecha_f'] = pd.to_datetime(Stock_CS_gb['Fecha_t'], format="%Y%m%d")
Stock_CS_gb['weekday'] = Stock_CS_gb['Fecha_f'].dt.dayofweek
#### FILTRO DIA LUNES
Stock_CS_gb_L = Stock_CS_gb[Stock_CS_gb["weekday"]==0]
Stock_CS_gb_L_f = Stock_CS_gb_L#pd.merge(Stock_CS_gb_L, Maestro_SKU_KO, on=["SKU_SAP"],how="outer")
Stock_CS_gb_L_f = Stock_CS_gb_L_f[["Fecha_t", "SKU_SAP", "SKU", "Stock_Cj", "HLs"]]
Stock_CS_gb_L_f = Stock_CS_gb_L_f.dropna(subset=['SKU_SAP','Fecha_t'])
#Stock_CS_gb_L_f.to_csv("D:/BI Plannig/02 ETL/Stock_ABI.txt")
Stock_CS_gb_L_f['Id'] = "ABI" + '-' + Stock_CS_gb_L_f.Fecha_t.astype(str) + '-' + Stock_CS_gb_L_f.SKU.astype(str)
Stock_CS_gb_L_f = Stock_CS_gb_L_f[["Id", "Stock_Cj"]]

STOCK = Stock_CS_gb_L_f.append(Stock_KOA_gb_L)
STOCK = STOCK.append(Stock_KOE_gb_L)

STOCK.to_csv("D:/BI Plannig/02 ETL/02 DATA/07 Stock/Stock.txt")
STOCK

In [ ]:
########################## FIN ###############################

In [ ]:
################### KOE ############
Stock_KOE_2['Id'] = "Embonor" +"-"+ Stock_KOE_2.Fecha_t.astype(str) +"-"+Stock_KOE_2.Sku.astype(str)
Stock_KOE_gb = Stock_KOE_2.groupby(by = ["Id"])["Stock_Cj"].sum().reset_index()
Stock_KOE_gb = pd.DataFrame(Stock_KOE_gb)
t = Stock_KOE_gb.Id.str.split("-", n=2, expand=True)
Stock_KOE_gb['Bottler'] = t[0]
Stock_KOE_gb['Fecha'] = t[1]
Stock_KOE_gb['Sku'] = t[2]

Stock_KOE_gb['Fecha_f'] = pd.to_datetime(Stock_KOE_gb['Fecha'], format="%Y%m%d")
Stock_KOE_gb['weekday'] = Stock_KOE_gb['Fecha_f'].dt.dayofweek
Stock_KOE_gb_L = Stock_KOE_gb[Stock_KOE_gb["weekday"]==0]
#Stock_KOE_gb_L = Stock_KOE_gb_L[["Id", "Stock_Cj"]]

Stock_KOE_gb_L.Stock_Cj = Stock_KOE_gb_L.Stock_Cj.astype(float)
Stock_KOE_gb_L.Stock_Cj = Stock_KOE_gb_L.Stock_Cj.round(3)
Stock_KOE_gb_L.Stock_Cj = Stock_KOE_gb_L.Stock_Cj*1000
Stock_KOE_gb_L.Stock_Cj = Stock_KOE_gb_L.Stock_Cj.round(0)

Stock_KOE_gb_L = Stock_KOE_gb_L[Stock_KOE_gb_L["Sku"]!="nan"]

Stock_KOE_gb_L.to_csv("D:/BI Plannig/02 ETL/02 DATA/07 Stock/Stock_KOE.txt")

In [ ]:
################ KOA ############
Stock_KOA.Sku = Stock_KOA.Sku*10
Stock_KOA.Sku = Stock_KOA.Sku.astype(str)
t = Stock_KOA.Sku.str.split(".", n=1, expand=True)
Stock_KOA.Sku = t[0]
Stock_KOA = Stock_KOA.drop(Stock_KOA[Stock_KOA['Sku']=='nan'].index)
Stock_KOA['Id'] = "Andina" +"-"+ Stock_KOA.Fecha_t.astype(str) +"-"+Stock_KOA.Sku.astype(str)
Stock_KOA_gb = Stock_KOA.groupby(by = ["Id"])["Stock_Cj"].sum().reset_index()
Stock_KOA_gb = pd.DataFrame(Stock_KOA_gb)
t = Stock_KOA_gb.Id.str.split("-", n=2, expand=True)
Stock_KOA_gb['Bottler'] = t[0]
Stock_KOA_gb['Fecha'] = t[1]
Stock_KOA_gb['Sku'] = t[2]
Stock_KOA_gb = Stock_KOA_gb[Stock_KOA_gb["Stock_Cj"]>0]

Stock_KOA_gb = Stock_KOA_gb.drop(Stock_KOA_gb[Stock_KOA_gb['Fecha']==' 092905'].index)
Stock_KOA_gb.Fecha = Stock_KOA_gb.Fecha.astype(float)
Stock_KOA_gb.Fecha = Stock_KOA_gb.Fecha.round(0)
Stock_KOA_gb = Stock_KOA_gb[Stock_KOA_gb["Fecha"]<20239999]
Stock_KOA_gb['Fecha_f'] = pd.to_datetime(Stock_KOA_gb['Fecha'], format="%Y%m%d")
Stock_KOA_gb['weekday'] = Stock_KOA_gb['Fecha_f'].dt.dayofweek
Stock_KOA_gb_L = Stock_KOA_gb[Stock_KOA_gb["weekday"]==0]
Stock_KOA_gb_L = Stock_KOA_gb_L[["Id", "Stock_Cj"]]
Stock_KOA_gb_L

Stock_KOA_gb_L.Stock_Cj = Stock_KOA_gb_L.Stock_Cj.astype(float)
Stock_KOA_gb_L.Stock_Cj = Stock_KOA_gb_L.Stock_Cj.round(3)
Stock_KOA_gb_L.Stock_Cj = Stock_KOA_gb_L.Stock_Cj*1000
Stock_KOA_gb_L.Stock_Cj = Stock_KOA_gb_L.Stock_Cj.round(0)
Stock_KOA_gb_L.to_csv("D:/BI Plannig/02 ETL/02 DATA/07 Stock/Stock_KOA.txt")

In [ ]:
Stock_KOA

In [ ]:
os.chdir('D:/Reporting/Maestras/')
Maestro_SKU = pd.read_excel('Materiales  Abi.xlsx', sheet_name='Hoja1', dtype ={'Cj/Pall simil':np.float64,'HL/cj':np.float64} ,decimal='.')
Maestro_SKU.rename(columns = {'Material': 'Sku_KOA','Material ABI': 'Sku','HL/cj':'HL_cj'} ,inplace=True)
Maestro_SKU.HL_cj = np.round(Maestro_SKU.HL_cj,5)
Maestro_SKU.HL_cj =Maestro_SKU.HL_cj/10
Maestro_SKU.HL_cj = np.round(Maestro_SKU.HL_cj,6)

################################################ STOCK KOA  ####################################################################
os.chdir('D:/Reporting/Bases Manuales/01 STOCK/Stock KOA')
Stock_KOA = []
csv_files = glob.glob('*.csv')

for filename in csv_files:
    data = pd.read_csv(filename,encoding='ANSI', sep=';',dtype={'Stock':np.float64,'Codigo CD': np.float64} ,decimal='.')
    data['filename']=filename
    Stock_KOA.append(data)
Stock_KOA = pd.concat(Stock_KOA,ignore_index=True)

Titulo =Stock_KOA[["filename"]]
Titulo = Titulo["filename"].str.split(" ", n=1, expand=True)
Stock_KOA["Fecha_t"]=Titulo[0].str.replace(r"-","")
Stock_KOA.rename(columns = {'Codigo CD': 'CD','SKU': 'Sku_KOA', 'Stock': 'Stock_Cj'} ,inplace=True)
Stock_KOA.Sku_KOA = Stock_KOA.Sku_KOA.astype(int)

Maestra_SKU_KOA = Maestro_SKU
Maestra_SKU_KOA.Sku_KOA = Maestra_SKU_KOA.Sku_KOA.astype(int)

Maestra_SKU_KOA = Maestro_SKU[["Sku_KOA","Sku"]]
Maestra_SKU_KOA = Maestra_SKU_KOA.drop_duplicates(subset=["Sku_KOA"])
Stock_KOA = pd.merge(Stock_KOA,Maestra_SKU_KOA,on=["Sku_KOA"],how="outer")
#Stock_KOA.CD = Stock_KOA.CD/10
Stock_KOA.Sku = Stock_KOA.Sku.astype(np.float64)

Stock_KOA["PRI"] =0
Stock_KOA=Stock_KOA[["CD","Sku","Stock_Cj","Fecha_t","PRI"]]
Stock_KOA = Stock_KOA.dropna(subset=['Fecha_t'])

Stock_KOA.Sku = Stock_KOA.Sku.astype(str)
t = Stock_KOA.Sku.str.split(".", n=1, expand=True)
Stock_KOA.Sku = t[0]
Stock_KOA = Stock_KOA.drop(Stock_KOA[Stock_KOA['Sku']=='nan'].index)

Stock_KOA['Id'] = "Andina" +"-"+ Stock_KOA.Fecha_t.astype(str) +"-"+Stock_KOA.Sku.astype(str)
Stock_KOA_gb = Stock_KOA.groupby(by = ["Id"])["Stock_Cj"].sum().reset_index()
Stock_KOA_gb = pd.DataFrame(Stock_KOA_gb)
t = Stock_KOA_gb.Id.str.split("-", n=2, expand=True)
Stock_KOA_gb['Bottler'] = t[0]
Stock_KOA_gb['Fecha'] = t[1]
Stock_KOA_gb['Sku'] = t[2]
Stock_KOA_gb = Stock_KOA_gb[Stock_KOA_gb["Stock_Cj"]>0]

Stock_KOA_gb = Stock_KOA_gb.drop(Stock_KOA_gb[Stock_KOA_gb['Fecha']==' 092905'].index)
Stock_KOA_gb.Fecha = Stock_KOA_gb.Fecha.astype(float)
Stock_KOA_gb.Fecha = Stock_KOA_gb.Fecha.round(0)
Stock_KOA_gb = Stock_KOA_gb[Stock_KOA_gb["Fecha"]<20239999]
Stock_KOA_gb['Fecha_f'] = pd.to_datetime(Stock_KOA_gb['Fecha'], format="%Y%m%d")
Stock_KOA_gb['weekday'] = Stock_KOA_gb['Fecha_f'].dt.dayofweek
Stock_KOA_gb_L = Stock_KOA_gb[Stock_KOA_gb["weekday"]==0]
Stock_KOA_gb_L = Stock_KOA_gb_L[["Id", "Stock_Cj"]]

Stock_KOA_gb_L.Stock_Cj = Stock_KOA_gb_L.Stock_Cj.astype(float)
Stock_KOA_gb_L.Stock_Cj = Stock_KOA_gb_L.Stock_Cj.round(3)
Stock_KOA_gb_L.Stock_Cj = Stock_KOA_gb_L.Stock_Cj*1000
Stock_KOA_gb_L.Stock_Cj = Stock_KOA_gb_L.Stock_Cj.round(0)
Stock_KOA_gb_L
Stock_KOA_gb_L.to_csv("D:/BI Plannig/02 ETL/02 DATA/07 Stock/Stock_KOA.txt")

In [ ]:
##### OTRO INTENTO ######

In [ ]:
Stock_ABI_gb = Stock_ABI1.groupby(by = ["Id"])["Stock_Cj"].sum().reset_index()
Stock_ABI_gb = pd.DataFrame(Stock_ABI_gb)
t = Stock_ABI_gb.Id.str.split("-", n=2, expand=True)
Stock_ABI_gb['Bottler'] = t[0]
Stock_ABI_gb['Fecha'] = t[1]
Stock_ABI_gb['Sku'] = t[2]
Stock_ABI_gb['Fecha_d'] = Stock_ABI_gb.Fecha.str.extract(r"(\d{2})") 
Stock_ABI_gb['Fecha_dm'] = Stock_ABI_gb.Fecha.str.extract(r"(\d{4})")
Stock_ABI_gb['Fecha_m'] = Stock_ABI_gb.Fecha_dm.apply(lambda x: x[2:])
Stock_ABI_gb['Fecha_A'] = Stock_ABI_gb.Fecha.apply(lambda x: x[4:]) 
Stock_ABI_gb['Fecha_t'] = Stock_ABI_gb.Fecha_A+Stock_ABI_gb.Fecha_m+Stock_ABI_gb.Fecha_d
Stock_ABI_gb['Fecha_f'] = pd.to_datetime(Stock_ABI_gb['Fecha_t'], format="%Y%m%d")
Stock_ABI_gb['weekday'] = Stock_ABI_gb['Fecha_f'].dt.dayofweek
Stock_ABI_gb_L = Stock_ABI_gb[Stock_ABI_gb["weekday"]==0]
Stock_ABI_gb_L

In [ ]:
Stock_ABI = pd.merge(Stock_ABI,Maestro_SKU_S,on=["Sku"],how="outer")
Stock_ABI = Stock_ABI.dropna(subset=['Simil','Fecha_t'])
Stock_ABI['Id'] = "ABI" +"-"+ Stock_ABI.Fecha_t.astype(str) +"-"+Stock_ABI.Simil.astype(str)

Stock_ABI_gb = Stock_ABI.groupby(by = ["Id"])["Stock_Cj"].sum().reset_index()
Stock_ABI_gb = pd.DataFrame(Stock_ABI_gb)
t = Stock_ABI_gb.Id.str.split("-", n=2, expand=True)
Stock_ABI_gb['Bottler'] = t[0]
Stock_ABI_gb['Fecha'] = t[1]
Stock_ABI_gb['Sku'] = t[2]

Stock_ABI_gb['Fecha_d'] = Stock_ABI_gb.Fecha.str.extract(r"(\d{2})") 
Stock_ABI_gb['Fecha_dm'] = Stock_ABI_gb.Fecha.str.extract(r"(\d{4})")
Stock_ABI_gb['Fecha_m'] = Stock_ABI_gb.Fecha_dm.apply(lambda x: x[2:])
Stock_ABI_gb['Fecha_A'] = Stock_ABI_gb.Fecha.apply(lambda x: x[4:]) 
Stock_ABI_gb['Fecha_t'] = Stock_ABI_gb.Fecha_A+Stock_ABI_gb.Fecha_m+Stock_ABI_gb.Fecha_d
Stock_ABI_gb['Fecha_f'] = pd.to_datetime(Stock_ABI_gb['Fecha_t'], format="%Y%m%d")
Stock_ABI_gb['weekday'] = Stock_ABI_gb['Fecha_f'].dt.dayofweek
Stock_ABI_gb_L = Stock_ABI_gb[Stock_ABI_gb["weekday"]==0]

Stock_ABI_gb['Id'] = "ABI"+"-"+Stock_ABI_gb.Fecha_A+Stock_ABI_gb.Fecha_m+Stock_ABI_gb.Fecha_d+"-"+Stock_ABI_gb.Sku
Stock_ABI_gb_L = Stock_ABI_gb_L[["Id", "Stock_Cj"]]
Stock_ABI_gb_L

In [ ]:
################## Este SI ################
os.chdir('D:/Reporting/Maestras/')
Maestro_SKU = pd.read_excel('Materiales  Abi.xlsx', sheet_name='Hoja1', dtype ={'Cj/Pall simil':np.float64,'HL/cj':np.float64} ,decimal='.')
Maestro_SKU.rename(columns = {'Material': 'Sku_KOA','Material ABI': 'Sku','HL/cj':'HL_cj'} ,inplace=True)
Maestro_SKU_S = Maestro_SKU[["Sku", "Simil"]]
Maestro_SKU_S.Sku = Maestro_SKU_S.Sku.astype(str)
Maestro_SKU_S.Simil = Maestro_SKU_S.Simil.astype(str)

################ KOA ############
Stock_KOA.Sku = Stock_KOA.Sku*10
Stock_KOA.Sku = Stock_KOA.Sku.astype(str)
t = Stock_KOA.Sku.str.split(".", n=1, expand=True)
Stock_KOA.Sku = t[0]
Stock_KOA = Stock_KOA.drop(Stock_KOA[Stock_KOA['Sku']=='nan'].index)
Stock_KOA['Id'] = "Andina" +"-"+ Stock_KOA.Fecha_t.astype(str) +"-"+Stock_KOA.Sku.astype(str)
Stock_KOA_gb = Stock_KOA.groupby(by = ["Id"])["Stock_Cj"].sum().reset_index()
Stock_KOA_gb = pd.DataFrame(Stock_KOA_gb)
t = Stock_KOA_gb.Id.str.split("-", n=2, expand=True)
Stock_KOA_gb['Bottler'] = t[0]
Stock_KOA_gb['Fecha'] = t[1]
Stock_KOA_gb['Sku'] = t[2]
Stock_KOA_gb = Stock_KOA_gb[Stock_KOA_gb["Stock_Cj"]>0]
Stock_KOA_gb = Stock_KOA_gb.drop(Stock_KOA_gb[Stock_KOA_gb['Fecha']==' 092905'].index)
Stock_KOA_gb.Fecha = Stock_KOA_gb.Fecha.astype(float)
Stock_KOA_gb.Fecha = Stock_KOA_gb.Fecha.round(0)
Stock_KOA_gb = Stock_KOA_gb[Stock_KOA_gb["Fecha"]<20239999]
Stock_KOA_gb['Fecha_f'] = pd.to_datetime(Stock_KOA_gb['Fecha'], format="%Y%m%d")
Stock_KOA_gb['weekday'] = Stock_KOA_gb['Fecha_f'].dt.dayofweek
Stock_KOA_gb_L = Stock_KOA_gb[Stock_KOA_gb["weekday"]==0]
Stock_KOA_gb_L = Stock_KOA_gb_L[["Id", "Stock_Cj"]]
Stock_KOA_gb_L
################### KOE ############
Stock_KOE_2['Id'] = "Embonor" +"-"+ Stock_KOE_2.Fecha_t.astype(str) +"-"+Stock_KOE_2.Sku.astype(str)
Stock_KOE_gb = Stock_KOE_2.groupby(by = ["Id"])["Stock_Cj"].sum().reset_index()
Stock_KOE_gb = pd.DataFrame(Stock_KOE_gb)
t = Stock_KOE_gb.Id.str.split("-", n=2, expand=True)
Stock_KOE_gb['Bottler'] = t[0]
Stock_KOE_gb['Fecha'] = t[1]
Stock_KOE_gb['Sku'] = t[2]
Stock_KOE_gb['Fecha_f'] = pd.to_datetime(Stock_KOE_gb['Fecha'], format="%Y%m%d")
Stock_KOE_gb['weekday'] = Stock_KOE_gb['Fecha_f'].dt.dayofweek
Stock_KOE_gb_L = Stock_KOE_gb[Stock_KOE_gb["weekday"]==0]
Stock_KOE_gb_L = Stock_KOE_gb_L[["Id", "Stock_Cj"]]
Stock_KOE_gb_L
################## ABI #############

#Stock_ABI = pd.merge(Stock_ABI,Maestro_SKU_S,on=["Sku"],how="outer")
#Stock_ABI = Stock_ABI.dropna(subset=['Simil','Fecha_t'])
#Stock_ABI['Id'] = "ABI" +"-"+ Stock_ABI.Fecha_t.astype(str) +"-"+Stock_ABI.Simil.astype(str)
#Stock_ABI_gb = Stock_ABI.groupby(by = ["Id"])["Stock_Cj"].sum().reset_index()
#Stock_ABI_gb = pd.DataFrame(Stock_ABI_gb)
#t = Stock_ABI_gb.Id.str.split("-", n=2, expand=True)
#Stock_ABI_gb['Bottler'] = t[0]
#Stock_ABI_gb['Fecha'] = t[1]
#Stock_ABI_gb['Sku'] = t[2]
Stock_ABI_gb['Fecha_d'] = Stock_ABI_gb.Fecha.str.extract(r"(\d{2})") 
Stock_ABI_gb['Fecha_dm'] = Stock_ABI_gb.Fecha.str.extract(r"(\d{4})")
Stock_ABI_gb['Fecha_m'] = Stock_ABI_gb.Fecha_dm.apply(lambda x: x[2:])
Stock_ABI_gb['Fecha_A'] = Stock_ABI_gb.Fecha.apply(lambda x: x[4:]) 
Stock_ABI_gb['Fecha_t'] = Stock_ABI_gb.Fecha_A+Stock_ABI_gb.Fecha_m+Stock_ABI_gb.Fecha_d
Stock_ABI_gb['Fecha_f'] = pd.to_datetime(Stock_ABI_gb['Fecha_t'], format="%Y%m%d")
Stock_ABI_gb['weekday'] = Stock_ABI_gb['Fecha_f'].dt.dayofweek
Stock_ABI_gb_L = Stock_ABI_gb[Stock_ABI_gb["weekday"]==0]

Stock_ABI_gb['Id'] = "ABI"+"-"+Stock_ABI_gb.Fecha_A+Stock_ABI_gb.Fecha_m+Stock_ABI_gb.Fecha_d+"-"+Stock_ABI_gb.Sku
Stock_ABI_gb_L = Stock_ABI_gb_L[["Id", "Stock_Cj"]]
Stock_ABI_gb


In [ ]:
Stock_ABI_gb = Stock_ABI
Stock_ABI_gb['Fecha_d'] = Stock_ABI_gb.Fecha_t.str.extract(r"(\d{2})") 
Stock_ABI_gb['Fecha_dm'] = Stock_ABI_gb.Fecha_t.str.extract(r"(\d{4})")
Stock_ABI_gb['Fecha_m'] = Stock_ABI_gb.Fecha_dm.apply(lambda x: x[2:])
Stock_ABI_gb['Fecha_A'] = Stock_ABI_gb.Fecha_t.apply(lambda x: x[4:]) 
Stock_ABI_gb['Fecha_am'] = Stock_ABI_gb.Fecha_A+Stock_ABI_gb.Fecha_m+Stock_ABI_gb.Fecha_d
Stock_ABI_gb['Fecha_f'] = pd.to_datetime(Stock_ABI_gb['Fecha_am'], format="%Y%m%d")
Stock_ABI_gb['weekday'] = Stock_ABI_gb['Fecha_f'].dt.dayofweek
Stock_ABI_gb_L = Stock_ABI_gb[Stock_ABI_gb["weekday"]==0]

Stock_ABI_gb['If'] = Stock_ABI_gb.Fecha_A+Stock_ABI_gb.Fecha_m+Stock_ABI_gb.Fecha_d

Stock_ABI_gb

In [ ]:
os.chdir('D:/Reporting/Maestras/')
Maestro_SKU = pd.read_excel('Materiales  Abi.xlsx', sheet_name='Hoja1', dtype ={'Cj/Pall simil':np.float64,'HL/cj':np.float64} ,decimal='.')
Maestro_SKU.rename(columns = {'Material': 'Sku_KOA','Material ABI': 'Sku','HL/cj':'HL_cj'} ,inplace=True)
Maestro_SKU_S = Maestro_SKU[["Sku", "Simil"]]
Maestro_SKU_S.Sku = Maestro_SKU_S.Sku.astype(str)
Maestro_SKU_S.Simil = Maestro_SKU_S.Simil.astype(str)

Stock_ABI = pd.merge(Stock_ABI,Maestro_SKU_S,on=["Sku"],how="outer")
Stock_ABI = Stock_ABI.dropna(subset=['Simil','Fecha_t'])

Stock_ABI['Id'] = "ABI" +"-"+ Stock_ABI.Fecha_t.astype(str) +"-"+Stock_ABI.Simil.astype(str)
Stock_ABI_gb = Stock_ABI.groupby(by = ["Id"])["Stock_Cj"].sum().reset_index()
Stock_ABI_gb = pd.DataFrame(Stock_ABI_gb)
t = Stock_ABI_gb.Id.str.split("-", n=2, expand=True)
Stock_ABI_gb['Bottler'] = t[0]
Stock_ABI_gb['Fecha'] = t[1]
Stock_ABI_gb['Sku'] = t[2]
Stock_ABI_gb['Fecha_d'] = Stock_ABI_gb.Fecha.str.extract(r"(\d{2})") 
Stock_ABI_gb['Fecha_dm'] = Stock_ABI_gb.Fecha.str.extract(r"(\d{4})")
Stock_ABI_gb['Fecha_m'] = Stock_ABI_gb.Fecha_dm.apply(lambda x: x[2:])
Stock_ABI_gb['Fecha_A'] = Stock_ABI_gb.Fecha.apply(lambda x: x[4:]) 

Stock_ABI_gb['Fecha_t'] = Stock_ABI_gb.Fecha_A+Stock_ABI_gb.Fecha_m+Stock_ABI_gb.Fecha_d
Stock_ABI_gb['Fecha_f'] = pd.to_datetime(Stock_ABI_gb['Fecha_t'], format="%Y%m%d")
Stock_ABI_gb['weekday'] = Stock_ABI_gb['Fecha_f'].dt.dayofweek
Stock_ABI_gb_L = Stock_ABI_gb[Stock_ABI_gb["weekday"]==0]


Stock_ABI_gb['Id'] = "ABI"+"-"+Stock_ABI_gb.Fecha_A+Stock_ABI_gb.Fecha_m+Stock_ABI_gb.Fecha_d+"-"+Stock_ABI_gb.Sku
Stock_ABI_gb_L = Stock_ABI_gb_L[["Id", "Stock_Cj"]]
Stock_ABI_gb

In [ ]:
Stock_ABI['Id'] = "ABI" +"-"+ Stock_ABI.Fecha_t.astype(str) +"-"+Stock_ABI.Sku.astype(str)
Stock_ABI_gb = Stock_ABI.groupby(by = ["Id"])["Stock_Cj"].sum().reset_index()
Stock_ABI_gb = pd.DataFrame(Stock_ABI_gb)
t = Stock_ABI_gb.Id.str.split("-", n=2, expand=True)
Stock_ABI_gb['Bottler'] = t[0]
Stock_ABI_gb['Fecha'] = t[1]
Stock_ABI_gb['Sku'] = t[2]
Stock_ABI_gb['Fecha_d'] = Stock_ABI_gb.Fecha.str.extract(r"(\d{2})") 
Stock_ABI_gb['Fecha_dm'] = Stock_ABI_gb.Fecha.str.extract(r"(\d{4})")
Stock_ABI_gb['Fecha_m'] = Stock_ABI_gb.Fecha_dm.apply(lambda x: x[2:])
Stock_ABI_gb['Fecha_A'] = Stock_ABI_gb.Fecha.apply(lambda x: x[4:]) 
Stock_ABI_gb['Id'] = "ABI"+"-"+Stock_ABI_gb.Fecha_A+Stock_ABI_gb.Fecha_m+Stock_ABI_gb.Fecha_d+"-"+Stock_ABI_gb.Sku
Stock_ABI_gb['Fecha_t'] = Stock_ABI_gb.Fecha_A+Stock_ABI_gb.Fecha_m+Stock_ABI_gb.Fecha_d
Stock_ABI_gb['Fecha_f'] = pd.to_datetime(Stock_ABI_gb['Fecha_t'], format="%Y%m%d")
Stock_ABI_gb['weekday'] = Stock_ABI_gb['Fecha_f'].dt.dayofweek
Stock_ABI_gb_L = Stock_ABI_gb[Stock_ABI_gb["weekday"]==0]


Maestro_SKU_S = Maestro_SKU[["Sku", "Simil"]]
Stock_ABI_gb_L = pd.merge(Stock_ABI_gb_L,Maestro_SKU_S,on=["Sku"],how="outer")
Stock_ABI_gb_L = Stock_ABI_gb_L.dropna(subset=['Simil','Fecha'])
#FC_OP_ok2["Id"] = FC_OP_ok2.Bottler +"-"+ FC_OP_ok2.Semana +"-"+ FC_OP_ok2.Simil

Stock_ABI_gb_L